# Model Evaluation: Comparing Forecasting Methods

This notebook evaluates the performance of different forecasting models:
- ARIMA
- SARIMA (Seasonal ARIMA)
- SARIMAX (Seasonal ARIMA with eXogenous variables)
- Prophet
- LSTM
- TCN (Temporal Convolutional Network)
- Transformer

We'll use data from **10 randomly selected companies** and compare model accuracy using multiple metrics.

## Model Input/Output Summary

### Input Format
All models accept the same input format:
- **Input**: `pd.Series` - A time series of values (e.g., daily returns) with a datetime index
- **Data Type**: Single univariate time series (one column of values over time)

### Output Format
All models return:
- **Forecast**: `pd.Series` - Predicted values for future periods with datetime index
- **Confidence Intervals**: `pd.Series` or `pd.DataFrame` (if available) - Upper and lower bounds

### Model-Specific Details

#### 1. ARIMA
- **Input**: Univariate time series (pd.Series)
- **Internal Processing**: Uses entire historical series to fit ARIMA model
- **Output**: 
  - Forecast: `pd.Series` with future dates
  - Confidence Intervals: `pd.DataFrame` with 'lower' and 'upper' columns

#### 2. SARIMA
- **Input**: Univariate time series (pd.Series)
- **Internal Processing**: Uses entire historical series, captures seasonal patterns (e.g., weekly patterns)
- **Output**: 
  - Forecast: `pd.Series` with future dates
  - Confidence Intervals: `pd.DataFrame` with 'lower' and 'upper' columns

#### 3. SARIMAX
- **Input**: Univariate time series (pd.Series) + optional exogenous variables
- **Internal Processing**: Uses historical series with additional features (rolling stats, lags)
- **Output**: 
  - Forecast: `pd.Series` with future dates
  - Confidence Intervals: `pd.DataFrame` with 'lower' and 'upper' columns

#### 4. Prophet
- **Input**: Univariate time series (pd.Series) - converts to DataFrame with 'ds' (dates) and 'y' (values)
- **Internal Processing**: Uses entire historical series, handles seasonality
- **Output**: 
  - Forecast: `pd.Series` with future dates
  - Confidence Intervals: `pd.DataFrame` with 'lower' and 'upper' columns

#### 5. LSTM
- **Input**: Univariate time series (pd.Series)
- **Internal Processing**: 
  - Creates sequences of `lookback_window` (default: 60) past values
  - Normalizes data using MinMaxScaler
  - Input shape: `(batch_size, lookback_window, 1)`
- **Output**: 
  - Forecast: `pd.Series` with future dates
  - Confidence Intervals: `None` (not available)

#### 6. TCN
- **Input**: Univariate time series (pd.Series)
- **Internal Processing**: 
  - Creates sequences of `lookback_window` (default: 60) past values
  - Normalizes data using MinMaxScaler
  - Uses dilated convolutions with causal padding
  - Input shape: `(batch_size, lookback_window, 1)`
- **Output**: 
  - Forecast: `pd.Series` with future dates
  - Confidence Intervals: `None` (not available)

#### 7. Transformer
- **Input**: Univariate time series (pd.Series)
- **Internal Processing**: 
  - Creates sequences of `lookback_window` (default: 60) past values
  - Normalizes data using MinMaxScaler
  - Uses multi-head attention mechanism
  - Input shape: `(batch_size, lookback_window, 1)`
- **Output**: 
  - Forecast: `pd.Series` with future dates
  - Confidence Intervals: `None` (not available)

### Key Differences
- **ARIMA, SARIMA, SARIMAX & Prophet**: Use entire historical series, provide confidence intervals
- **LSTM, TCN, Transformer**: Use sliding windows of past values, no confidence intervals
- **All models**: Return forecasts as `pd.Series` with datetime index matching the input frequency
- **SARIMA**: Extends ARIMA with seasonal components (useful for weekly/monthly patterns)
- **SARIMAX**: Extends SARIMA with exogenous variables (additional features like rolling statistics)


In [1]:
# Import necessary libraries
import sys
import os
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from typing import Dict, List, Tuple
import random

# Fix for TensorFlow/protobuf compatibility issue
# This error occurs when there's a version mismatch between TensorFlow and protobuf
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# Suppress TensorFlow warnings and protobuf errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

# Try to fix protobuf issue by setting environment variable before importing TensorFlow
try:
    import tensorflow as tf
    # Suppress the specific protobuf error
    import logging
    logging.getLogger('tensorflow').setLevel(logging.ERROR)
except ImportError:
    pass

# Add project root to path
# In Jupyter notebooks, Path.cwd() gives the current working directory
# We assume the notebook is run from the project root or evaluation/ directory
current_dir = Path.cwd()
if current_dir.name == 'evaluation':
    project_root = current_dir.parent
else:
    # If running from project root, use current directory
    project_root = current_dir

sys.path.insert(0, str(project_root))

# Import forecast functions
from src.forecast import (
    arima_forecast,
    sarima_forecast,
    sarimax_forecast,
    prophet_forecast,
    lstm_forecast,
    tcn_forecast,
    transformer_forecast
)

print("Libraries imported successfully!")
print(f"Project root: {project_root}")
print(f"Current directory: {current_dir}")
print("\nNote: If you encounter protobuf errors with TensorFlow models,")
print("try running: pip install protobuf==3.20.3")


2025-12-07 00:18:35.049265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765041515.066721  836874 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765041515.071827  836874 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765041515.086159  836874 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765041515.086180  836874 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765041515.086182  836874 computation_placer.cc:177] computation placer alr

Libraries imported successfully!
Project root: /home/dang.cpm/__MY_SPACE__/VinUni/FinLove
Current directory: /home/dang.cpm/__MY_SPACE__/VinUni/FinLove/evaluation

Note: If you encounter protobuf errors with TensorFlow models,
try running: pip install protobuf==3.20.3


In [2]:
# Fix protobuf compatibility issue
# This cell fixes the common TensorFlow/protobuf version mismatch error
import subprocess
import sys

def fix_protobuf():
    """Attempt to fix protobuf version compatibility."""
    try:
        # Try to install compatible protobuf version
        print("Attempting to fix protobuf compatibility...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "protobuf==3.20.3"], 
                             stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print("✓ Protobuf version fixed. Please restart the kernel and run all cells again.")
        return True
    except Exception as e:
        print(f"Could not automatically fix protobuf. Please run manually:")
        print(f"  pip install protobuf==3.20.3")
        print(f"Or: pip install --upgrade protobuf")
        return False

# Uncomment the line below if you're getting protobuf errors
# fix_protobuf()

# Alternative: Set environment variable to use Python implementation
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

print("Protobuf compatibility settings applied.")
print("If errors persist, restart kernel after running: pip install protobuf==3.20.3")


Protobuf compatibility settings applied.
If errors persist, restart kernel after running: pip install protobuf==3.20.3


## Step 1: Load and Prepare Data

We'll randomly select 10 companies and load their data.


In [3]:
# Define data directory
data_dir = project_root / 'data'

# Get all CSV files (excluding cache directory)
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# Randomly select 10 companies
random.seed(42)  # For reproducibility
num_companies = min(10, len(csv_files))  # Use 10 or all available if less than 10
selected_companies = random.sample(csv_files, num_companies)

print(f"Selected {num_companies} companies: {[f.split('_')[0] for f in selected_companies]}")
print(f"Total files available: {len(csv_files)}")

def load_company_data(filepath: Path) -> Tuple[str, pd.Series]:
    """
    Load company data from CSV file.
    
    The CSV structure is:
    - Row 1: Column names (Price, Close, High, Low, Open, Volume)
    - Row 2: Ticker metadata
    - Row 3: Date header row
    - Row 4+: Actual data with dates in first column
    
    Args:
        filepath: Path to the CSV file
        
    Returns:
        Tuple of (ticker, Series with Date index and Close prices)
    """
    # Read CSV with header from first row, skip rows 2 and 3 (ticker and Date header)
    df = pd.read_csv(filepath, header=0, skiprows=[1, 2])
    
    # Extract ticker from filename
    ticker = filepath.stem.split('_')[0]
    
    # The first column should be 'Price' which contains the dates
    # Rename it to 'Date' for clarity
    if 'Price' in df.columns:
        df = df.rename(columns={'Price': 'Date'})
    
    # Convert Date column to datetime and set as index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # Use Close price column
    if 'Close' not in df.columns:
        # If Close column doesn't exist, check what columns we have
        raise ValueError(f"Close column not found. Available columns: {df.columns.tolist()}")
    
    close_prices = df['Close'].copy()
    
    return ticker, close_prices

# Load data for both companies
company_data = {}
for filename in selected_companies:
    filepath = data_dir / filename
    ticker, prices = load_company_data(filepath)
    company_data[ticker] = prices
    print(f"\n{ticker}:")
    print(f"  Data points: {len(prices)}")
    print(f"  Date range: {prices.index[0]} to {prices.index[-1]}")
    print(f"  Price range: ${prices.min():.2f} - ${prices.max():.2f}")


Selected 10 companies: ['MA', 'PG', 'XLK', 'NKE', 'XLB', 'HD', 'XLV', 'AAPL', 'AMZN', 'XLY']
Total files available: 29

MA:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00:00
  Price range: $76.10 - $598.17

PG:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00:00
  Price range: $56.85 - $175.07

XLK:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00:00
  Price range: $34.66 - $304.13

NKE:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00:00
  Price range: $44.32 - $167.31

XLB:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00:00
  Price range: $30.68 - $95.72

HD:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00:00
  Price range: $88.72 - $423.60

XLV:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00:00
  Price range: $54.01 - $154.61

AAPL:
  Data points: 2512
  Date range: 2015-11-27 00:00:00 to 2025-11-21 00:00

## Step 2: Calculate Returns and Split Data

We'll calculate daily returns and split the data into training and testing sets.


In [4]:
# Calculate returns for each company
# Returns = (Price_t - Price_{t-1}) / Price_{t-1}
returns_data = {}
for ticker, prices in company_data.items():
    returns = prices.pct_change().dropna()
    returns_data[ticker] = returns
    print(f"\n{ticker} Returns:")
    print(f"  Data points: {len(returns)}")
    print(f"  Mean return: {returns.mean():.6f}")
    print(f"  Std return: {returns.std():.6f}")
    print(f"  Min return: {returns.min():.6f}")
    print(f"  Max return: {returns.max():.6f}")

# Split data into train and test sets
# Use 80% for training, 20% for testing
train_test_split = {}
for ticker, returns in returns_data.items():
    split_idx = int(len(returns) * 0.8)
    train_returns = returns.iloc[:split_idx]
    test_returns = returns.iloc[split_idx:]
    
    train_test_split[ticker] = {
        'train': train_returns,
        'test': test_returns
    }
    
    print(f"\n{ticker} Split:")
    print(f"  Train: {len(train_returns)} points ({train_returns.index[0]} to {train_returns.index[-1]})")
    print(f"  Test: {len(test_returns)} points ({test_returns.index[0]} to {test_returns.index[-1]})")



MA Returns:
  Data points: 2511
  Mean return: 0.000844
  Std return: 0.016934
  Min return: -0.127254
  Max return: 0.166109

PG Returns:
  Data points: 2511
  Mean return: 0.000454
  Std return: 0.011794
  Min return: -0.087373
  Max return: 0.120090

XLK Returns:
  Data points: 2511
  Mean return: 0.000891
  Std return: 0.015279
  Min return: -0.138140
  Max return: 0.134257

NKE Returns:
  Data points: 2511
  Mean return: 0.000220
  Std return: 0.019837
  Min return: -0.199809
  Max return: 0.155315

XLB Returns:
  Data points: 2511
  Mean return: 0.000421
  Std return: 0.013125
  Min return: -0.110084
  Max return: 0.117601

HD Returns:
  Data points: 2511
  Mean return: 0.000588
  Std return: 0.015489
  Min return: -0.197938
  Max return: 0.137508

XLV Returns:
  Data points: 2511
  Mean return: 0.000426
  Std return: 0.010513
  Min return: -0.098610
  Max return: 0.077057

AAPL Returns:
  Data points: 2511
  Mean return: 0.001094
  Std return: 0.018354
  Min return: -0.128647
 

## Step 3: Define Evaluation Metrics

We'll calculate multiple metrics to compare model performance:
- MAE (Mean Absolute Error)
- RMSE (Root Mean Squared Error)
- MAPE (Mean Absolute Percentage Error)
- R² (Coefficient of Determination)


In [5]:
def calculate_metrics(y_true: pd.Series, y_pred: pd.Series) -> Dict[str, float]:
    """
    Calculate evaluation metrics for forecast predictions.
    
    Args:
        y_true: True values
        y_pred: Predicted values
        
    Returns:
        Dictionary with metric names and values
    """
    # Align indices in case they don't match exactly
    common_idx = y_true.index.intersection(y_pred.index)
    if len(common_idx) == 0:
        # If no common index, use positional alignment
        min_len = min(len(y_true), len(y_pred))
        y_true_aligned = y_true.iloc[:min_len].values
        y_pred_aligned = y_pred.iloc[:min_len].values
    else:
        y_true_aligned = y_true.loc[common_idx].values
        y_pred_aligned = y_pred.loc[common_idx].values
    
    # Calculate metrics
    mae = np.mean(np.abs(y_true_aligned - y_pred_aligned))
    rmse = np.sqrt(np.mean((y_true_aligned - y_pred_aligned) ** 2))
    
    # MAPE - handle division by zero
    mask = np.abs(y_true_aligned) > 1e-10
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true_aligned[mask] - y_pred_aligned[mask]) / y_true_aligned[mask])) * 100
    else:
        mape = np.nan
    
    # R² score
    ss_res = np.sum((y_true_aligned - y_pred_aligned) ** 2)
    ss_tot = np.sum((y_true_aligned - np.mean(y_true_aligned)) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan
    
    return {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R²': r2
    }

print("Evaluation metrics function defined!")


Evaluation metrics function defined!


## Step 4: Evaluate Models

We'll evaluate each model on all 10 companies and collect the results.

**Note**: This may take a while as we're evaluating 7 models × 10 companies = 70 model runs.


In [6]:
# Define forecast horizon (number of days to predict)
forecast_horizon = 30

# Store results for each company and model
results = {}

# Store ARIMA forecasts and actual values for visualization
arima_forecasts = {}
arima_actuals = {}
arima_conf_intervals = {}

# Track progress
total_companies = len(train_test_split)
company_num = 0

# Evaluate each company
for ticker, data in train_test_split.items():
    company_num += 1
    print(f"\n{'='*60}")
    print(f"Evaluating models for {ticker} ({company_num}/{total_companies})")
    print(f"{'='*60}")
    
    train_returns = data['train']
    test_returns = data['test']
    
    # Limit test set to forecast_horizon for fair comparison
    actual_test = test_returns.iloc[:forecast_horizon]
    
    results[ticker] = {}
    
    # 1. ARIMA
    print(f"\n[1/7] ARIMA...")
    try:
        forecast, conf_int = arima_forecast(
            train_returns,
            forecast_horizon=forecast_horizon,
            auto_select=True
        )
        metrics = calculate_metrics(actual_test, forecast)
        results[ticker]['ARIMA'] = metrics
        
        # Store forecasts for visualization
        arima_forecasts[ticker] = forecast
        arima_actuals[ticker] = actual_test
        arima_conf_intervals[ticker] = conf_int
        
        print(f"  ✓ ARIMA completed - MAE: {metrics['MAE']:.6f}, RMSE: {metrics['RMSE']:.6f}")
    except Exception as e:
        print(f"  ✗ ARIMA failed: {str(e)}")
        results[ticker]['ARIMA'] = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'R²': np.nan}
    
    # 2. SARIMA
    print(f"\n[2/7] SARIMA...")
    try:
        forecast, _ = sarima_forecast(
            train_returns,
            forecast_horizon=forecast_horizon,
            auto_select=True
        )
        metrics = calculate_metrics(actual_test, forecast)
        results[ticker]['SARIMA'] = metrics
        print(f"  ✓ SARIMA completed - MAE: {metrics['MAE']:.6f}, RMSE: {metrics['RMSE']:.6f}")
    except Exception as e:
        print(f"  ✗ SARIMA failed: {str(e)}")
        results[ticker]['SARIMA'] = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'R²': np.nan}
    
    # 3. SARIMAX
    print(f"\n[3/7] SARIMAX...")
    try:
        forecast, _ = sarimax_forecast(
            train_returns,
            forecast_horizon=forecast_horizon,
            auto_select=True
        )
        metrics = calculate_metrics(actual_test, forecast)
        results[ticker]['SARIMAX'] = metrics
        print(f"  ✓ SARIMAX completed - MAE: {metrics['MAE']:.6f}, RMSE: {metrics['RMSE']:.6f}")
    except Exception as e:
        print(f"  ✗ SARIMAX failed: {str(e)}")
        results[ticker]['SARIMAX'] = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'R²': np.nan}
    
    # # 4. Prophet
    # print(f"\n[4/7] Prophet...")
    # try:
    #     forecast, _ = prophet_forecast(
    #         train_returns,
    #         forecast_horizon=forecast_horizon,
    #         yearly_seasonality=False,  # Disable for daily returns
    #         weekly_seasonality=True,
    #         daily_seasonality=False
    #     )
    #     metrics = calculate_metrics(actual_test, forecast)
    #     results[ticker]['Prophet'] = metrics
    #     print(f"  ✓ Prophet completed - MAE: {metrics['MAE']:.6f}, RMSE: {metrics['RMSE']:.6f}")
    # except Exception as e:
    #     print(f"  ✗ Prophet failed: {str(e)}")
    #     results[ticker]['Prophet'] = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'R²': np.nan}
    
    # # 5. LSTM
    # print(f"\n[5/7] LSTM...")
    # try:
    #     forecast, _ = lstm_forecast(
    #         train_returns,
    #         forecast_horizon=forecast_horizon,
    #         lookback_window=60,
    #         lstm_units=50,
    #         epochs=20,  # Reduced for faster evaluation
    #         batch_size=32,
    #         use_cache=True,
    #         ticker=ticker
    #     )
    #     metrics = calculate_metrics(actual_test, forecast)
    #     results[ticker]['LSTM'] = metrics
    #     print(f"  ✓ LSTM completed - MAE: {metrics['MAE']:.6f}, RMSE: {metrics['RMSE']:.6f}")
    # except Exception as e:
    #     print(f"  ✗ LSTM failed: {str(e)}")
    #     results[ticker]['LSTM'] = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'R²': np.nan}
    
    # # 6. TCN
    # print(f"\n[6/7] TCN...")
    # try:
    #     forecast, _ = tcn_forecast(
    #         train_returns,
    #         forecast_horizon=forecast_horizon,
    #         lookback_window=60,
    #         num_filters=64,
    #         kernel_size=3,
    #         num_blocks=2,
    #         epochs=20,  # Reduced for faster evaluation
    #         batch_size=32,
    #         use_cache=True,
    #         ticker=ticker
    #     )
    #     metrics = calculate_metrics(actual_test, forecast)
    #     results[ticker]['TCN'] = metrics
    #     print(f"  ✓ TCN completed - MAE: {metrics['MAE']:.6f}, RMSE: {metrics['RMSE']:.6f}")
    # except Exception as e:
    #     print(f"  ✗ TCN failed: {str(e)}")
    #     results[ticker]['TCN'] = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'R²': np.nan}
    
    # # 7. Transformer
    # print(f"\n[7/7] Transformer...")
    # try:
    #     forecast, _ = transformer_forecast(
    #         train_returns,
    #         forecast_horizon=forecast_horizon,
    #         lookback_window=60,
    #         d_model=64,
    #         num_heads=4,
    #         num_layers=2,
    #         epochs=20,  # Reduced for faster evaluation
    #         batch_size=32,
    #         use_cache=True,
    #         ticker=ticker
    #     )
    #     metrics = calculate_metrics(actual_test, forecast)
    #     results[ticker]['Transformer'] = metrics
    #     print(f"  ✓ Transformer completed - MAE: {metrics['MAE']:.6f}, RMSE: {metrics['RMSE']:.6f}")
    # except Exception as e:
    #     print(f"  ✗ Transformer failed: {str(e)}")
    #     results[ticker]['Transformer'] = {'MAE': np.nan, 'RMSE': np.nan, 'MAPE': np.nan, 'R²': np.nan}
    
    print(f"\n{ticker} evaluation completed!")

print(f"\n{'='*60}")
print("All evaluations completed!")
print(f"{'='*60}")



Evaluating models for MA (1/10)

[1/7] ARIMA...
  ✓ ARIMA completed - MAE: 0.005169, RMSE: 0.007730

[2/7] SARIMA...
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.59655D+00    |proj g|=  8.79598D-01

At iterate    5    f= -2.59663D+00    |proj g|=  1.17913D-01


 This problem is unconstrained.



At iterate   10    f= -2.59675D+00    |proj g|=  3.14134D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     14     17      1     0     0   1.885D-03  -2.597D+00
  F =  -2.5967607544813247     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.43246D+00    |proj g|=  1.60454D+01

At iterate    5    f= -2.50277D+00    |proj g|=  8.33986D+00

At iterate   10    f= -2.56553D+00    |proj g|=  4.19992D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     13     42      1     0     0   7.653D+01  -2.568D+00
  F =  -2.5675771654343174     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


...

 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      4     23      1     0     0   1.217D-02  -3.077D+00
  F =  -3.0774479093009646     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.90476D+00    |proj g|=  2.50799D+01

At iterate    5    f= -2.95415D+00    |proj g|=  4.79559D-01


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Visualization of ARIMA, SARIMA, SARIMAX vs. Ground Truth for all companies
for ticker in results.keys():
    plt.figure(figsize=(12, 6))

    # Plot actual test (ground truth)
    actual = arima_actuals.get(ticker)
    if actual is not None:
        plt.plot(actual.index, actual.values, marker="o", label="Ground Truth", color="black", linestyle='-')

    # ARIMA prediction & confidence interval
    arima_f = arima_forecasts.get(ticker)
    arima_ci = arima_conf_intervals.get(ticker)
    if arima_f is not None:
        plt.plot(arima_f.index, arima_f.values, label="ARIMA Prediction", color="#1f77b4", linestyle='--')
        if arima_ci is not None:
            # Try to robustly handle multiindex or different CI column names
            ci_cols = list(arima_ci.columns)
            lower_col = None
            upper_col = None

            colnames_lower = [c for c in ci_cols if "low" in c.lower()]
            colnames_upper = [c for c in ci_cols if "upp" in c.lower()]

            if len(colnames_lower) == 1 and len(colnames_upper) == 1:
                lower_col = colnames_lower[0]
                upper_col = colnames_upper[0]
            # fallback if only first two columns:
            elif len(ci_cols) >= 2:
                lower_col = ci_cols[0]
                upper_col = ci_cols[1]

            if lower_col and upper_col:
                plt.fill_between(
                    arima_ci.index,
                    arima_ci[lower_col],
                    arima_ci[upper_col],
                    color="#1f77b4",
                    alpha=0.15,
                    label="ARIMA 95% CI"
                )

    # SARIMA prediction
    sarima_f = None
    try:
        sarima_f, _ = sarima_forecast(
            train_test_split[ticker]["train"],
            forecast_horizon=len(actual),
            auto_select=True
        )
    except Exception:
        sarima_f = None
    if sarima_f is not None:
        plt.plot(sarima_f.index, sarima_f.values, label="SARIMA Prediction", color="#ff7f0e", linestyle='--')

    # SARIMAX prediction
    sarimax_f = None
    try:
        sarimax_f, _ = sarimax_forecast(
            train_test_split[ticker]["train"],
            forecast_horizon=len(actual),
            auto_select=True
        )
    except Exception:
        sarimax_f = None
    if sarimax_f is not None:
        plt.plot(sarimax_f.index, sarimax_f.values, label="SARIMAX Prediction", color="#2ca02c", linestyle='--')

    plt.title(f"{ticker} Forecast vs Ground Truth\n(ARIMA, SARIMA, SARIMAX)")
    plt.xlabel("Date")
    plt.ylabel("Return")
    plt.legend(loc="upper left")
    plt.tight_layout()
    plt.grid(True, alpha=0.25)
    plt.show()


## Step 6: Create Comparison Table

Now we'll create a comprehensive comparison table showing all metrics for all models.


In [ ]:
# Create a comprehensive results table
all_results = []

# Collect results for each company and model
for ticker, model_results in results.items():
    for model_name, metrics in model_results.items():
        all_results.append({
            'Company': ticker,
            'Model': model_name,
            'MAE': metrics['MAE'],
            'RMSE': metrics['RMSE'],
            'MAPE': metrics['MAPE'],
            'R²': metrics['R²']
        })

# Create DataFrame
results_df = pd.DataFrame(all_results)

# Display the results table
print("\n" + "="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)
print("\nDetailed Results:")
print(results_df.to_string(index=False))

# Create a summary table with averages across companies
print("\n" + "="*80)
print("AVERAGE METRICS ACROSS ALL COMPANIES")
print("="*80)

summary_data = []
for model in ['ARIMA', 'SARIMA', 'SARIMAX', 'Prophet', 'LSTM', 'TCN', 'Transformer']:
    model_data = results_df[results_df['Model'] == model]
    if len(model_data) > 0:
        summary_data.append({
            'Model': model,
            'MAE (avg)': model_data['MAE'].mean(),
            'RMSE (avg)': model_data['RMSE'].mean(),
            'MAPE (avg)': model_data['MAPE'].mean(),
            'R² (avg)': model_data['R²'].mean(),
            'MAE (std)': model_data['MAE'].std(),
            'RMSE (std)': model_data['RMSE'].std()
        })

summary_df = pd.DataFrame(summary_data)
print("\nSummary Statistics:")
print(summary_df.to_string(index=False))


## Step 6: Formatted Comparison Table

A nicely formatted table for easy comparison.


In [ ]:
# Create a pivot table for better visualization
pivot_mae = results_df.pivot(index='Company', columns='Model', values='MAE')
pivot_rmse = results_df.pivot(index='Company', columns='Model', values='RMSE')
pivot_r2 = results_df.pivot(index='Company', columns='Model', values='R²')

print("\n" + "="*80)
print("MAE (Mean Absolute Error) - Lower is Better")
print("="*80)
print(pivot_mae.round(6).to_string())

print("\n" + "="*80)
print("RMSE (Root Mean Squared Error) - Lower is Better")
print("="*80)
print(pivot_rmse.round(6).to_string())

print("\n" + "="*80)
print("R² (Coefficient of Determination) - Higher is Better (closer to 1)")
print("="*80)
print(pivot_r2.round(4).to_string())

# Find best model for each metric
print("\n" + "="*80)
print("BEST MODEL FOR EACH METRIC (across all companies)")
print("="*80)

# Calculate average metrics
avg_metrics = results_df.groupby('Model').agg({
    'MAE': 'mean',
    'RMSE': 'mean',
    'R²': 'mean'
}).round(6)

best_mae = avg_metrics['MAE'].idxmin()
best_rmse = avg_metrics['RMSE'].idxmin()
best_r2 = avg_metrics['R²'].idxmax()

print(f"\nBest MAE:  {best_mae} ({avg_metrics.loc[best_mae, 'MAE']:.6f})")
print(f"Best RMSE: {best_rmse} ({avg_metrics.loc[best_rmse, 'RMSE']:.6f})")
print(f"Best R²:   {best_r2} ({avg_metrics.loc[best_r2, 'R²']:.4f})")

# Overall ranking (based on average rank across all metrics)
avg_metrics['MAE_rank'] = avg_metrics['MAE'].rank()
avg_metrics['RMSE_rank'] = avg_metrics['RMSE'].rank()
avg_metrics['R²_rank'] = avg_metrics['R²'].rank(ascending=False)  # Higher is better
avg_metrics['Overall_Rank'] = (avg_metrics['MAE_rank'] + avg_metrics['RMSE_rank'] + avg_metrics['R²_rank']) / 3

print("\n" + "="*80)
print("OVERALL RANKING (lower rank is better)")
print("="*80)
ranking = avg_metrics[['MAE', 'RMSE', 'R²', 'Overall_Rank']].sort_values('Overall_Rank')
print(ranking.to_string())


## Summary

This evaluation compares the performance of 7 forecasting models:
- **ARIMA**: Statistical time series model
- **SARIMA**: Seasonal ARIMA model (captures seasonal patterns)
- **SARIMAX**: Seasonal ARIMA with exogenous variables (uses additional features)
- **Prophet**: Facebook's forecasting tool
- **LSTM**: Long Short-Term Memory neural network
- **TCN**: Temporal Convolutional Network
- **Transformer**: Attention-based neural network

### Metrics Explained:
- **MAE (Mean Absolute Error)**: Average absolute difference between predicted and actual values. Lower is better.
- **RMSE (Root Mean Squared Error)**: Square root of average squared differences. Penalizes large errors more. Lower is better.
- **MAPE (Mean Absolute Percentage Error)**: Average percentage error. Lower is better.
- **R² (Coefficient of Determination)**: Proportion of variance explained. Closer to 1 is better.

The models are evaluated on daily returns data, which is more challenging to predict than prices due to the noisy nature of financial returns.
